In [1]:
api_key = "INSIRA SUA API KEY DA BINANCE AQUI"
secret_key = "INSIRA SUA SECRETY KEY DA BINANCE AQUI"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from binance.client import Client
from functools import reduce
from sklearn.cluster import KMeans
import talib
import yfinance  as yf
import plotly.graph_objects as go

In [3]:
client = Client(api_key=api_key, api_secret = secret_key, tld = "com")

In [4]:
#Obtendo todos os pares USDT disponíveis
prices = pd.DataFrame(client.get_all_tickers())
prices = prices[prices.symbol.str.contains("USDT")]
#prices = prices[:20]
symbolcoins = prices['symbol'].tolist()


In [ ]:
listadf = []
adj_lista = []
adj_volatility = []
for symbol in symbolcoins:
    ###Esse Loop interage com API para retornar dados históricos (200 dias)
     ## de todos os pares USDT disponíveis na Binance e retorna dois dataframes

    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, "200 day ago UTC")
  
    timestamp = client._get_earliest_valid_timestamp(symbol = symbol, interval = "1d")
    pd.to_datetime(timestamp, unit = "ms")
    df = pd.DataFrame(klines)
    #Filtrando inconsistências
    if len(df.columns) > 13 or len(df.columns) < 5:
          continue
    
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close",
                  "Volume", "Clos Time", "Quote Asset Volume", "Number of Trade",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    data = df[["Date", "Close", "Volume"]].set_index("Date")
    for column in data.columns: data[column] = pd.to_numeric(data[column], errors = "coerce")
    
    data['Volume'] = data['Volume'] *  data['Close'] #Volume em USD
     
    data[f"vol_ch_{symbol}"] = np.log(data.Volume.div(data.Volume.shift(1))) #Calculando diferença de Volume d-1
    data["price_ch"] = (data['Close'] - data['Close'].shift(1)) / data['Close'].shift(1) #Calculando diferença de preço d-1
    adj_volatility.append(str(symbol))
    #Calculando desvio padrão 30 períodos
    data['std30'] = data['price_ch'].rolling(30, min_periods=1).std()*(30**.5)
    adj_volatility.append(data['std30'].iloc[-1]) 
    data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')
    adj_volatility.append((data['Volume'].ewm(span=7, adjust=False).mean())[-1]) #Volume media 7dias
    #Calculado volume medio, atribuindo maior peso aos valores mais recentes
    adj_volatility.append(data['price_ch'].iloc[-1])
    data.rename(columns={'Close': f'Close_{symbol}', 'Volume': f'Volume_{symbol}',
                         'price_ch' : f'price_ch_{symbol}'}, inplace= True)
    
    adj_lista.append(adj_volatility)
    listadf.append(data)
    
    adj_volatility = []

In [ ]:
df=  reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), listadf)
prices_changes = df.filter(regex='price_ch_')
prices_changes = prices_changes*100
prices_changes

In [ ]:
#Filtrando alterações de volume e volatilidade ajustada pela mediana
adjdf= pd.DataFrame(adj_lista, columns=['symbol', 'adj', 'Volume', 'price_ch'])
adj = adjdf[(adjdf['Volume'] > adjdf['Volume'].median()) & (adjdf['adj'] > adjdf['adj'].median())]
adj


In [ ]:
adjdf

## Volatilidade Ajustada Vs Volume - Pares USDT

In [ ]:
z = np.random.rand(100)
data = [go.Scatter(x=adj['Volume'], y=adj['adj'], text=adj['symbol'], mode='markers',  
                   marker=dict( color=z, colorscale='Viridis'))]

layout = go.Layout(title='Volatilidade Ajustada vs Volume',
                   xaxis=dict(title='Volume 7D'),
                   yaxis=dict(title='Volatilidade Ajustada (Método Desvio Padrão 30 períodos)'))

fig = go.Figure(data=data, layout=layout)
fig.show()


In [ ]:
#Para facilitar a visualizacao dos ativos, criou-se um modelo de clusterizacao com base na volatilidade e alteracao de preco (d-1)

# Selecionando as colunas de interesse
X = adj[["adj", "price_ch"]]

# Definindo o número de clusters desejado
n_clusters = 4

# Instanciando o modelo k-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Treinando o modelo
kmeans.fit(X)

# Visualizando as classes atribuídas pelo modelo
labels = kmeans.labels_

adj['cluster'] = labels

# Plotando o gráfico com as classes atribuídas pelo modelo
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=labels)
plt.xlabel("Volatility")
plt.ylabel("Pct_change")
plt.show()

In [ ]:
for cluster in  adj['cluster'].unique():
    adj_cluster = adj[(adj['cluster']== cluster)]
    adj_cluster = adj_cluster.sort_values('adj')
    langs = round(adj_cluster['adj'], 1), adj_cluster['symbol']
    price_ch = round(adj_cluster['price_ch']*100, 2)
    data = [go.Bar(
       x = langs,
       y = price_ch
                )]
    layout = go.Layout(title=f'Ativo, Volatilidade Ajustada vs Alteração de Preços(D-1), CLUSTER: {cluster}',
                   xaxis=dict(title='Ativo e Volatilidade Ajustada'),
                   yaxis=dict(title='Alteração de preço'))

    fig = go.Figure(data=data, layout=layout)
    fig.show()

## Matrix de correlação quanto a paridade de alteração de preco em relacao ao BTC


In [ ]:
#Filtrando ativos alavancados e derivativos

ativos = prices_changes.loc[:, ~prices_changes.columns.str.contains("DOWN") & 
                            ~prices_changes.columns.str.contains("UP")]


# calcular a correlação entre cada ativo e o BTC
btc_corr = ativos.drop('price_ch_BTCUSDT', axis=1).corrwith(ativos['price_ch_BTCUSDT'], axis=0)
df_corr = pd.DataFrame(btc_corr, columns=['corr'])
df_corr.sort_values('corr', inplace=True)
#LISTANDO ATIVOS QUE TEM COMPORTAMENTO INVERSO AO BTC
df_corr[:30]

In [ ]:
df_corr.sort_values('corr', ascending=False).iloc[:20]

## Variação de preço em relação a D-1(%)

In [ ]:

# Criando os dados
data = dict(
    type='treemap',
    labels= prices_changes.columns.tolist() + (prices_changes.iloc[-1,:].tolist()),
    parents=['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 
            '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
            '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 
            '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 
             '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
             '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
             '', '', '', '', '', 
             '', '', '', '', '', '' ],
    values= prices_changes.iloc[-1,:].tolist(),
    branchvalues='total'
)

# Criando o gráfico
fig = go.Figure(data=[data])


# Exibindo o gráfico
fig.show()

In [ ]:
def calculo_volume(symbol):
    timestamp = client._get_earliest_valid_timestamp(symbol = symbol, interval = "1d")
    pd.to_datetime(timestamp, unit = "ms")
    bars= client.get_historical_klines(symbol = symbol, interval = "1d", start_str = timestamp, limit = 1000)
    bars
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Clos Time", "Quote Asset Volume", "Number of Trade", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    data = df [["Date", "Close", "Volume", "Taker Buy Quote Asset Volume", "Taker Buy Base Asset Volume","High", "Low"]].copy()
    data.set_index("Date", inplace = True)
    for column in data.columns: data[column] = pd.to_numeric(data[column], errors = "coerce")
    data['sma_30_Quote'] = (data["Taker Buy Quote Asset Volume"] * data['Close']).rolling(window=30).mean()
    data['sma_30_Base'] = (data["Taker Buy Base Asset Volume"] * data['Close']).rolling(window=30).mean()
    data["vol_ch"] = np.log(data.Volume.div(data.Volume.shift(1)))
    data["price_ch"] = np.log(data.Volume.div(data.Close.shift(1)))
    data['volatility'] = data['Close'].pct_change().std()
    return data

In [ ]:
symbol= 'BTCUSDT'
data = calculo_volume(symbol)


data

## Relação preço  vs volume de compra  (USD)

In [ ]:

fig = go.Figure()

# Adicionando dados ao gráfico
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name=f'Preço {symbol} USD'))
fig.add_trace(go.Scatter(x=data.index, y=data['sma_30_Quote'], name=f'Volume de Compra Bid price {symbol} (SMA 30 )', yaxis='y2'))

# Configurando o segundo eixo y
fig.update_layout(yaxis2=dict(title='Vol. Compra - Ordens Limit (SMA 30 )', overlaying='y', side='right'))

# Mostrando o gráfico
fig.show()

In [ ]:
fig = go.Figure()

# Adicionando dados ao gráfico
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name=f'Preço {symbol} USD'))
fig.add_trace(go.Scatter(x=data.index, y=data['sma_30_Base'], name='Volume de Compra a Mercado(SMA 30 dias)', yaxis='y2'))

# Configurando o segundo eixo y
fig.update_layout(yaxis2=dict(title='Vol. Compra - Ordes Disparadas a Mercado (SMA 30 dias)', overlaying='y', side='right'))

# Mostrando o gráfico
fig.show()

## Retornos BTCUSD vs Principais índices (normalizado)

In [ ]:

def normalization(symbol, start, end):
    df = yf.download(symbol, start, end)
    close = df.Close.copy().sort_index()
    close.index = close.index.strftime('%d/%m/%Y')
    close_norm = close.copy()
    for i in close.columns:
           close_norm[i] = (close[i]/close[i].dropna().iloc[0]).mul(100)
    close_norm = close_norm.groupby(level=0).sum().replace(0, np.nan)
    #close_norm = pd.to_datetime(close_norm, '%d/%m/%Y')
    close_norm.index = pd.to_datetime(close_norm.index, dayfirst=True)
    close_norm = close_norm.sort_index()
    return close_norm




In [ ]:
import datetime as dt
start = "2023-01-01"
end = dt.date.today()
symbol = ['^GSPC', "^DJI", "^IXIC", '^HSI', '^KS11', "BTC-USD"]
close_norm= normalization(symbol, start, end)
dict_label = {'BTC-USD':'BTC', '^DJI': 'Dow Jones Industrial Average', '^GSPC':'S&P 500', '^HSI':'HANG SENG INDEX',
              '^IXIC': 'Índice Nasdaq', '^KS11': 'KOSPI Composite Index' }


close_norm

### ^IXIC - Índice Nasdaq
### ^DJI -  Dow Jones Industrial Average
### ^GSPC - S&P 500
### ^HSI -  HANG SENG INDEX (Hong Kong)
### ^KS11 - KOSPI Composite Index  (Coreia do Sul)

In [ ]:
  
close_norm.dropna().plot(figsize = (15, 8), fontsize = 13, logy = False)


plt.legend(dict_label, fontsize = 13)
plt.show()